In [ ]:
import glob

In [1]:
import numpy as np

G_E = 978032.7       # 赤道重力 (mGal)
k = 0.0053024        # 常數項
k_prime = 0.0000059  # 次常數項

FAA_GRADIENT = 0.3086       # 自由空氣梯度 (mGal/m)
RHO_CRUST = 2.67            # 地殼密度 (g/cm^3)
RHO_WATER = 1.03            # 海水密度 (g/cm^3)
BOUGUER_CONSTANT = 0.04192  # Bouguer 常數 (mGal/(g/cm^3 * m))

LATITUDE = 48.1195     # 緯度 (°N)
H_ELEVATION = 487.9    # 高度或水深 (m)
G_OBS = 980717.39      # 觀測重力 (mGal)


# === 計算函數 ===
def theoretical_gravity(phi_deg: float) -> float:
    """理論重力公式（簡化 WGS84）
    g = G_E * (1 + k*sin^2(phi) - k'*sin^4(phi))
    """
    phi = np.deg2rad(phi_deg)
    s2 = np.sin(phi)**2
    s4 = s2**2
    return G_E * (1 + k * s2 - k_prime * s4)


def free_air_correction(H: float) -> float:
    """自由空氣校正 FAC = 0.3086 * H"""
    return FAA_GRADIENT * H


def bouguer_plate_correction(H: float, rho: float) -> float:
    """布格校正 BC = 0.04192 * ρ * H"""
    return BOUGUER_CONSTANT * rho * H


# === 結果輸出函式 ===
def print_result(title: str, data: dict):
    print(f"\n--- {title} ---")
    for k, v in data.items():
        print(f"{k:45s}: {v:10.3f} mGal")


# === (a) 陸地站點 ===
def compute_land_station():
    H = H_ELEVATION
    g_theo = theoretical_gravity(LATITUDE)
    fac = free_air_correction(H)
    bc = bouguer_plate_correction(H, RHO_CRUST)

    faa = G_OBS - g_theo + fac
    ba = faa - bc

    print_result("陸地站點 (Land Station)", {
        "Theoretical gravity (g_theo)": g_theo,
        "Free-air correction (FAC)": fac,
        "Bouguer correction (BC, ρ=2.67)": bc,
        "Free-air anomaly (FAA)": faa,
        "Bouguer anomaly (BA)": ba
    })


# === (b) 海洋站點 ===
def compute_ocean_station():
    D = H_ELEVATION
    g_theo = theoretical_gravity(LATITUDE)
    fac = 0  # 觀測在海平面
    faa = G_OBS - g_theo + fac

    rho_contrast = RHO_WATER - RHO_CRUST
    bc_ocean = bouguer_plate_correction(D, rho_contrast)
    ba_ocean = faa + bc_ocean

    print_result("海洋站點 (Ocean Station)", {
        "Free-air anomaly (FAA)": faa,
        "Ocean Bouguer correction (ρw-ρc)": bc_ocean,
        "Bouguer anomaly (BA)": ba_ocean
    })


# === 主程式 ===
if __name__ == "__main__":
    print("🌍 Gravity Correction and Anomaly Calculation\n")
    compute_land_station()
    compute_ocean_station()
    print("\n✅ 計算完成。")


🌍 Gravity Correction and Anomaly Calculation


--- 陸地站點 (Land Station) ---
Theoretical gravity (g_theo)                 : 980905.680 mGal
Free-air correction (FAC)                    :    150.566 mGal
Bouguer correction (BC, ρ=2.67)              :     54.609 mGal
Free-air anomaly (FAA)                       :    -37.724 mGal
Bouguer anomaly (BA)                         :    -92.333 mGal

--- 海洋站點 (Ocean Station) ---
Free-air anomaly (FAA)                       :   -188.290 mGal
Ocean Bouguer correction (ρw-ρc)             :    -33.543 mGal
Bouguer anomaly (BA)                         :   -221.832 mGal

✅ 計算完成。
